In [16]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [23]:
EXPERIMENT = 'dec9_big_game_2'
MODEL = 'lstm_deep'
EXPERIMENT_DIR = f'/shared/outputs/{EXPERIMENT}/{MODEL}'
CHECKPOINT_NAME = 'checkpoint_1000'

In [26]:
reward_files = glob.glob(EXPERIMENT_DIR + f'/evaluations/rewards_{CHECKPOINT_NAME}*.pkl')

In [27]:
reward_files

['/shared/outputs/dec9_big_game_2/lstm_deep/evaluations/rewards_checkpoint_1000_checkpoint_1000_br_1.pkl',
 '/shared/outputs/dec9_big_game_2/lstm_deep/evaluations/rewards_checkpoint_1000.pkl',
 '/shared/outputs/dec9_big_game_2/lstm_deep/evaluations/rewards_checkpoint_1000_checkpoint_1000_br_0.pkl']

In [51]:
expected_values = dict()
for reward_file in reward_files:
    with open(reward_file, 'rb') as f:
        rewards = pickle.load(f)
        
    for player, values in rewards['rewards'].items():
        br_agent = rewards['br_agent']
        if br_agent is None:
            key = player
        else:
            key = f'{player}_{br_agent}'
        expected_values[key] = np.array(values).mean()

deltas = dict()
players = [k for k in expected_values.keys() if isinstance(k, int)]
for player in players:
    deltas[player] = expected_values[f'{player}_{player}'] - expected_values[player]
print(deltas)
print("Nash Conv estimate:", sum(list(deltas.values())))

{0: -23.235829274310028, 1: 18.310422548900007}
Nash Conv estimate: -4.925406725410021
